# 6장. 팩터 전략

In [2]:
# coding: utf-8
from typing import Dict, Optional

import pandas as pd
import plotly.express as px

from data.data_loader import PykrxDataLoader
from simulation.account import Account
from simulation.broker import Broker
from simulation.metric import cagr, mdd, sharpe_ratio, sortino_ratio
from simulation.utility import get_lookback_fromdate, rebalance

## 1. 팩터 투자

### 1.1. 유틸리티 함수 정의

In [3]:
def date_adjust(index_df: pd.DataFrame, df: pd.DataFrame):
    new_index = downsample_df(index_df).index
    # 원래 인덱스에 새 인덱스 삽입
    df.index = df.index.map(lambda x: x.replace(day=new_index[
        (new_index.year == x.year) & (new_index.month == x.month)][0].day))

    return df


def downsample_df(df: pd.DataFrame) -> pd.DataFrame:
    data = df.copy()
    data.index = pd.to_datetime(data.index)
    data['date'] = data.index
    data = data.resample('M').apply('last')

    return data.set_index(pd.DatetimeIndex(data.date)).drop(columns=['date'])

### 1.2. 팩터 전략 정의하기

#### 1.2.1. 모멘텀(상대) 전략 정의하기

In [44]:
def calculate_momentum(ohlcv_data: pd.DataFrame,
                       lookback_period: int,
                       skip_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(모멘텀)
    momentum_data = data.shift(periods=skip_period).rolling(
        window=lookback_period).apply(lambda x: x[-1] / x[0] - 1)

    return momentum_data

#### 1.2.2. 가치(PER, PBR) 및 배당 전략 정의하기

In [45]:
def calculate_fundamental(ohlcv_data: pd.DataFrame,
                          market_cap_data: pd.DataFrame,
                          fundamental_data: pd.DataFrame,
                          strategy_name: str,
                          lookback_period: int = 12, ) -> pd.DataFrame:
    # 룩백 길이 변환
    if ohlcv_data.frequency == 'd': lookback_period = int(lookback_period / 21)

    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 조정 및 재구조화(기본)
    fundamental_data = fundamental_adjuster(fundamental_data=fundamental_data,
                                            market_cap_data=market_cap_data)
    mapping = {'per': 'EPS', 'pbr': 'BPS', 'dividend': 'DPS'}
    target_fundamental = mapping.get(strategy_name)
    fundamental_data = fundamental_data[
        ['date', target_fundamental, 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)[target_fundamental]

    # 펀더멘탈 데이터 날짜 수정
    fundamental_data = date_adjust(index_df=data, df=fundamental_data)

    # 팩터 계산(PER, PBR, 배당)
    if strategy_name == "per":
        fundamental_data = fundamental_data.rolling(
            window=lookback_period).sum()
        fundamental_data = data / fundamental_data
    elif strategy_name == 'pbr':
        fundamental_data = data / fundamental_data
    elif strategy_name == 'dividend':
        fundamental_data = fundamental_data / data
    else:
        raise ValueError

    return fundamental_data


def fundamental_adjuster(fundamental_data: pd.DataFrame,
                         market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 현재 주식 수로 나누기
    market_cap_data['shares_div'] = market_cap_data['shares'].div(
        market_cap_data.groupby('ticker')['shares'].transform('last'))
    market_cap_data = market_cap_data[['shares_div', 'ticker']].reset_index()

    # 조정
    data = fundamental_data.reset_index().merge(
        market_cap_data, on=['date', 'ticker']).set_index(['date'])
    data['EPS'] = data['shares_div'] * data['EPS']
    data['BPS'] = data['shares_div'] * data['BPS']
    data['DPS'] = data['shares_div'] * data['DPS']

    # 주식 수 바뀐 당일 오류 탐색
    changed_row = data.copy()
    changed_row['previous'] = changed_row.shares_div.shift(1)
    changed_row = changed_row[changed_row.shares_div != changed_row.previous]
    changed_row = changed_row.reset_index().set_index(['ticker', 'date'])

    # 당일 오류 정정
    data = data.reset_index().set_index(['ticker', 'date'])
    for i, index in enumerate(data.index):
        if index in changed_row.index:
            data.loc[index] = data.iloc[i + 1]

    return data.reset_index()[['ticker', 'date', 'BPS', 'EPS', 'DPS']]

#### 1.2.3. 소형주 전략 정의하기

In [6]:
def calculate_small(ohlcv_data: pd.DataFrame,
                    market_cap_data: pd.DataFrame) -> pd.DataFrame:
    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 재구조화(주식 수)
    market_cap_data = market_cap_data[['shares',
                                       'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['shares']

    # 시장 데이터 날짜 수정
    market_cap_data = date_adjust(index_df=data, df=market_cap_data)

    # 팩터 계산(시가총액)
    market_cap_data = market_cap_data * data

    return market_cap_data

#### 1.2.4. 로우볼 전략 정의하기

In [49]:
def calculate_lowvol(ohlcv_data: pd.DataFrame,
                     lookback_period: int) -> pd.DataFrame:
    # 데이터 재구조화
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 팩터 계산(표준편차)
    std_data = data.pct_change().rolling(lookback_period).std()

    return std_data

#### 1.2.5. 수급 주체에 따른 투자 전략 정의하기

In [8]:
def calculate_trader(ohlcv_data: pd.DataFrame,
                     market_cap_data: pd.DataFrame,
                     trader_data: pd.DataFrame,
                     strategy_name: str,
                     lookback_period: Optional[int] = 1) -> pd.DataFrame:
    # 룩백 길이 변환
    if ohlcv_data.frequency == 'd': lookback_period = int(lookback_period / 21)

    # 데이터 재구조화(OHLCV)
    data = ohlcv_data[['close', 'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['close']

    # 데이터 재구조화(주식 수)
    market_cap_data = market_cap_data[['shares',
                                       'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)['shares']

    # 데이터 재구조화(수급 주체)
    trader_data = trader_data[[strategy_name,
                               'ticker']].reset_index().set_index(
        ['ticker', 'date']).unstack(level=0)[strategy_name]
    trader_data = trader_data.rolling(window=lookback_period).sum()

    # 시장 및 수급 주체 데이터 날짜 수정
    market_cap_data = date_adjust(index_df=data, df=market_cap_data)
    trader_data = date_adjust(index_df=data, df=trader_data)

    # 팩터 계산(수급 주체)
    market_cap_data = market_cap_data * data
    trader_data = trader_data / market_cap_data

    return trader_data

#### 1.2.6. 멀티 팩터 투자 전략 정의하기

In [9]:
def calculate_multifactor(ohlcv_data: pd.DataFrame,
                          oracle: bool = False) -> pd.DataFrame:
    # 포트폴리오 csv 가져오기
    filename = 'factor/ticker_weight_real.csv' if oracle else 'factor/ticker_weight_pred.csv'

    # 형태 조정
    premade = pd.read_csv(filename, index_col=0)
    premade.ticker = premade.ticker.astype(str).str.zfill(6)
    premade = premade.set_index(['ticker', 'date']).unstack(level=0).weight
    premade.index = pd.to_datetime(premade.index)

    # 데이터 크기 맞추기
    date_pad = downsample_df(ohlcv_data).drop(columns=ohlcv_data.columns)
    padded_premade = pd.concat([date_pad, premade])
    padded_premade = padded_premade[
        ~padded_premade.index.duplicated(keep='last')]

    return padded_premade.sort_index()

### 1.3. 시뮬레이션을 위한 함수 정의하기

In [10]:
def get_factor_weight(factor_data: pd.DataFrame,
                      buying_ratio: float,
                      strategy_name: str) -> Optional[Dict]:
    # 데이터 중 결측치가 있는지 확인함
    if factor_data.isnull().values.any():
        return None

    # 매수 주식 선정
    reverse = {'per', 'pbr', 'small', 'lovwol'}
    ratio = buying_ratio if strategy_name in reverse else 1 - buying_ratio
    top_quantile = factor_data.quantile(ratio)
    if strategy_name in reverse:
        stocks_to_buy = factor_data[factor_data <= top_quantile].index.to_list()
    else:
        stocks_to_buy = factor_data[factor_data >= top_quantile].index.to_list()

    # 주식 비율 할당
    weights = 1 / len(stocks_to_buy) if stocks_to_buy else 0
    portfolio = {ticker: weights if ticker in stocks_to_buy else 0.0 for ticker
                 in factor_data.index}

    return portfolio

### 1.4. 팩터 투자 전략을 이용한 시뮬레이션 함수 정의하기

In [46]:
def simulate_factor(ohlcv_data: pd.DataFrame,
                    market_cap_data: Optional[pd.DataFrame],
                    fundamental_data: Optional[pd.DataFrame],
                    trader_data: Optional[pd.DataFrame],
                    lookback_period: Optional[int],
                    skip_period: Optional[int],
                    strategy_name: str,
                    buying_ratio: float = 0.1) -> Account:
    # 계좌 및 브로커 선언
    account = Account(initial_cash=100000000)
    broker = Broker()

    # 팩터 계산
    if strategy_name == 'relative':
        factor_data = calculate_momentum(ohlcv_data=ohlcv_data,
                                         lookback_period=lookback_period,
                                         skip_period=skip_period, )
    elif strategy_name in {'per', 'pbr', 'dividend'}:
        factor_data = calculate_fundamental(ohlcv_data=ohlcv_data,
                                            market_cap_data=market_cap_data,
                                            fundamental_data=fundamental_data,
                                            lookback_period=lookback_period,
                                            strategy_name=strategy_name)
    elif strategy_name == 'small':
        factor_data = calculate_small(ohlcv_data=ohlcv_data,
                                      market_cap_data=market_cap_data)
    elif strategy_name in {'individual', 'institutional', 'foreign'}:
        factor_data = calculate_trader(ohlcv_data=ohlcv_data,
                                       market_cap_data=market_cap_data,
                                       trader_data=trader_data,
                                       lookback_period=lookback_period,
                                       strategy_name=strategy_name)
    elif strategy_name == 'lowvol':
        factor_data = calculate_lowvol(ohlcv_data=ohlcv_data,
                                       lookback_period=lookback_period)
    elif strategy_name == 'multifactor':
        factor_data = calculate_multifactor(ohlcv_data=ohlcv_data)
    else:
        raise ValueError

    # 월별 리밸런싱 날짜 추출
    month_end = downsample_df(ohlcv_data).index

    for date, ohlcv in ohlcv_data.groupby(['date']):
        # 주문 집행 및 계좌 갱신
        transactions = broker.process_order(dt=date, data=ohlcv,
                                            orders=account.orders)
        account.update_position(transactions=transactions)
        account.update_portfolio(dt=date, data=ohlcv)
        account.update_order()

        # 리밸런싱 날짜가 아닐 경우 넘어가기
        if date not in month_end:
            continue

        # 팩터 전략을 이용하여 포트폴리오 구성
        factor_data_slice = factor_data.loc[date]
        weights = get_factor_weight(factor_data=factor_data_slice,
                                    buying_ratio=buying_ratio,
                                    strategy_name=strategy_name)

        print(f'Portfolio: {weights}')
        if weights is None:
            continue

        # 포트폴리오 비율 갱신
        account.update_weight(dt=date, weight=weights)

        # 주문 생성
        rebalance(dt=date, data=ohlcv, account=account, weights=weights)

    # 리밸런싱 날짜만 남기기
    account.account_history = [item for item in account.account_history if
                               item['date'].date() in month_end.date]

    return account

### 1.5. 데이터 불러오기

In [12]:
# 데이터 시작과 끝 날짜 정의
fromdate = '2013-04-01'
todate = '2021-12-31'

# 투자할 종목 후보 정의
ticker_list = ['000660', '005490', '051910', '006400', '005380', '000270',
               '012330', '068270', '105560', '096770', '055550', '066570',
               '047050', '032830', '015760', '086790', '000810', '033780',
               '034730', '034020', '009150', '138040', '010130', '001570',
               '010950', '024110', '030200', '051900', '009830', '086280',
               '011170', '011070', '012450', '036570', '005830', '161390',
               '034220', '004020', '032640', '097950', '000720', '006800',
               '006260', '010620', '011780', '078930', '005940', '029780',
               '128940', '035250', '016360', '021240', '010120', '052690',
               '008770', '071050', '000990', '001450', '020150', '039490',
               '111770', '000880', '004370', '036460', '007070', '138930',
               '139480', ]

# 데이터 불러오기
data_loader = PykrxDataLoader(fromdate=fromdate, todate=todate, market="KOSPI")
ohlcv_data = data_loader.load_stock_data(ticker_list=ticker_list, freq='m',
                                         delay=1)
ohlcv_data_day = data_loader.load_stock_data(ticker_list=ticker_list, freq='d',
                                             delay=1)
fundamental_data = data_loader.load_fundamental_data(ticker_list=ticker_list,
                                                     freq='m', delay=1)
market_cap_data = data_loader.load_market_cap_data(ticker_list=ticker_list,
                                                   freq='m', delay=1)
trader_data = data_loader.load_trader_data(ticker_list=ticker_list, freq='m',
                                           delay=1)

# 데이터 확인하기
print(f'주가 데이터: {ohlcv_data.shape}, 기본 데이터: {fundamental_data.shape}, '
      f'주식 수 데이터: {market_cap_data.shape}, 수급 주체 데이터: {trader_data.shape}')

주가 데이터: (7035, 7), 기본 데이터: (7035, 7), 주식 수 데이터: (7035, 5), 수급 주체 데이터: (7035, 6)


### 1.6. 전략 실행하기

In [13]:
# 시뮬레이션 시작일
simulation_fromdate = '2017-02-01'

In [16]:
def df_slicer(df: pd.DataFrame, fromdate: str):
    frequency = df.frequency if hasattr(df, 'frequency') else None
    sliced_df = df[fromdate:]
    sliced_df.__setattr__('frequency', frequency)

    return sliced_df

#### 1.6.1. 상대모멘텀 전략 실행하기

In [47]:
# 룩백 기간, 생략 기간 정의
lookback = 3
offset = 1
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'relative'

# 상대모멘텀 전략 실행하기
account_relative = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.14285714285714285, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.14285714285714285, '006800': 0.14285714285714285, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.14285714285714285, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.14285714285714285, '052690': 0.0, '055550': 0.0, '066570': 0.14285714285714285, '068270': 0.0, '0

#### 1.6.2. PER 가치 전략 실행하기

In [51]:
# 룩백 기간, 생략 기간 정의
lookback = 12
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'per'

# PER 가치 전략 실행하기
account_per = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=df_slicer(df=fundamental_data, fromdate=total_lookback),
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: {'000270': 0.14285714285714285, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.14285714285714285, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.14285714285714285, '012450': 0.0, '015760': 0.14285714285714285, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.14285714285714285, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0

#### 1.6.3. PBR 가치 전략 실행하기

In [20]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'pbr'

# PBR 가치 전략 실행하기
account_pbr = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=df_slicer(df=fundamental_data, fromdate=total_lookback),
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.14285714285714285, '004370': 0.0, '005380': 0.0, '005490': 0.14285714285714285, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.14285714285714285, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.14285714285714285, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.14285714285714285, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.14285714285714285, '096770': 0.0, '097950

#### 1.6.4. 배당 전략 실행하기

In [21]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'dividend'

# 배당 전략 실행하기
account_dividend = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=df_slicer(df=fundamental_data, fromdate=total_lookback),
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.14285714285714285, '006260': 0.0, '006400': 0.0, '006800': 0.14285714285714285, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.14285714285714285, '016360': 0.0, '020150': 0.0, '021240': 0.14285714285714285, '024110': 0.0, '029780': 0.14285714285714285, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.14285714285714285, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.1428571428

#### 1.6.5. 소형주 전략 실행하기

In [22]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'small'

# 소형주 전략 실행하기
account_small = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.14285714285714285, '010130': 0.0, '010620': 0.14285714285714285, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.14285714285714285, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950

#### 1.6.6. 로우볼 전략 실행하기

In [50]:
# 룩백 기간, 생략 기간 정의
lookback = 60
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='d')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'lowvol'

# 로우볼 전략 실행하기
account_lowvol = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=lookback,
    skip_period=offset,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: None
Portfolio: None
Portfolio: None
Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.14285714285714285, '012330': 0.0, '012450': 0.14285714285714285, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.14285714285714285, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '0867

#### 1.6.7. 개인 매수 따른 투자 전략 실행하기

In [24]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'individual'

# 수급 주체(개인) 전략 실행하기
account_individual = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=None,
    trader_data=df_slicer(df=trader_data, fromdate=total_lookback),
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.14285714285714285, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.14285714285714285, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.14285714285714285, '010120': 0.0, '010130': 0.14285714285714285, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.14285714285714285, '034220': 0.14285714285714285, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.14285714285714285, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '09677

#### 1.6.8. 기관 매수에 따른 투자 전략 실행하기

In [25]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'institutional'

# 수급 주체(기관) 전략 실행하기
account_institutional = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=None,
    trader_data=df_slicer(df=trader_data, fromdate=total_lookback),
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.14285714285714285, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.0, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.14285714285714285, '009150': 0.0, '009830': 0.0, '010120': 0.14285714285714285, '010130': 0.0, '010620': 0.14285714285714285, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950': 0.0, '105560': 0.0, '111770':

#### 1.6.9. 외국 매수에 따른 투자 전략 실행하기

In [26]:
# 룩백 기간, 생략 기간 정의
lookback = 1
offset = 0
total_lookback = get_lookback_fromdate(fromdate=simulation_fromdate,
                                       lookback=lookback + offset, freq='m')

# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'foreign'

# 수급 주체(외국인) 전략 실행하기
account_foreign = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=total_lookback),
    market_cap_data=df_slicer(df=market_cap_data, fromdate=total_lookback),
    fundamental_data=None,
    trader_data=df_slicer(df=trader_data, fromdate=total_lookback),
    lookback_period=lookback * 21,
    skip_period=offset * 21,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.14285714285714285, '001450': 0.0, '001570': 0.0, '004020': 0.14285714285714285, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.14285714285714285, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.14285714285714285, '011170': 0.0, '011780': 0.0, '012330': 0.0, '012450': 0.0, '015760': 0.0, '016360': 0.0, '020150': 0.0, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.0, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.14285714285714285, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950': 0.0, '105560'

#### 1.6.10. 멀티 팩터 투자 전략 실행하기

In [27]:
# 매수 비율 정의
ratio = 0.1

# 전략 정의
strategy = 'multifactor'

# 상대모멘텀 전략 실행하기
account_multifactor = simulate_factor(
    ohlcv_data=df_slicer(df=ohlcv_data_day, fromdate=simulation_fromdate),
    market_cap_data=None,
    fundamental_data=None,
    trader_data=None,
    lookback_period=None,
    skip_period=None,
    strategy_name=strategy,
    buying_ratio=ratio)

/var/folders/j2/p6vmvrcs1kg7j9c8811510sc0000gn/T/ipykernel_87974/888467401.py:44: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for date, ohlcv in ohlcv_data.groupby(['date']):


Portfolio: {'000270': 0.0, '000660': 0.0, '000720': 0.0, '000810': 0.0, '000880': 0.0, '000990': 0.0, '001450': 0.0, '001570': 0.14285714285714285, '004020': 0.0, '004370': 0.0, '005380': 0.0, '005490': 0.0, '005830': 0.0, '005940': 0.0, '006260': 0.0, '006400': 0.0, '006800': 0.0, '007070': 0.0, '008770': 0.0, '009150': 0.0, '009830': 0.0, '010120': 0.0, '010130': 0.0, '010620': 0.0, '010950': 0.0, '011070': 0.0, '011170': 0.0, '011780': 0.14285714285714285, '012330': 0.0, '012450': 0.14285714285714285, '015760': 0.0, '016360': 0.0, '020150': 0.14285714285714285, '021240': 0.0, '024110': 0.0, '029780': 0.0, '030200': 0.0, '032640': 0.0, '032830': 0.0, '033780': 0.0, '034020': 0.0, '034220': 0.0, '034730': 0.0, '035250': 0.0, '036460': 0.0, '036570': 0.14285714285714285, '039490': 0.0, '047050': 0.0, '051900': 0.0, '051910': 0.0, '052690': 0.0, '055550': 0.0, '066570': 0.0, '068270': 0.0, '071050': 0.0, '078930': 0.0, '086280': 0.0, '086790': 0.0, '096770': 0.0, '097950': 0.0, '105560'

### 1.7. 투자 결과 시각화하기

In [52]:
kospi = data_loader.load_index_data(ticker_list=['1001'], freq='m', delay=1)
kospi_returns = kospi.loc[simulation_fromdate:]
kospi_returns = kospi_returns['close'] * (100000000 / kospi_returns['close'][0])
kospi_returns = date_adjust(index_df=ohlcv_data_day, df=kospi_returns)
kospi_returns.name = 'kospi_return'
kospi_returns.index.name = 'date'

In [53]:
def metric_calculate(df: pd.DataFrame, fromdate: str):
    df = df.squeeze().loc[fromdate:]
    df_pct = df.pct_change()
    df_pct.iloc[0] = 0.0
    return f"CAGR: {cagr(returns=df_pct, freq='m')}, " \
           f"MDD: {mdd(returns=df_pct)}, " \
           f"Sharpe Ratio: {sharpe_ratio(returns=df_pct, freq='m')}, " \
           f"Sortino Ratio: {sortino_ratio(returns=df_pct, freq='m')}"

In [62]:
# 총자산 정보 가져오기
total_asset1 = pd.DataFrame(account_relative.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset2 = pd.DataFrame(account_per.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset3 = pd.DataFrame(account_pbr.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset4 = pd.DataFrame(account_dividend.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset5 = pd.DataFrame(account_small.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset6 = pd.DataFrame(account_lowvol.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset7 = pd.DataFrame(account_individual.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset8 = pd.DataFrame(account_institutional.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset9 = pd.DataFrame(account_foreign.account_history)[
                   ['date', 'total_asset']].set_index('date')[
               simulation_fromdate:]
total_asset10 = pd.DataFrame(account_multifactor.account_history)[
    ['date', 'total_asset']].set_index('date')

# 자산 정보 결합하기
total_assets = pd.concat(
    [kospi_returns, total_asset1, total_asset2, total_asset3, total_asset4,
     total_asset5, total_asset6, total_asset7, total_asset8, total_asset9,
     total_asset10
     ],
    axis=1)
total_assets.columns = ["코스피", "상대모멘텀", "PER(가치)", "PBR(가치)", "배당",
                        "소형주", "로우볼", "개인", "기관", "외국인", '멀티팩터'
                        ]

# 자산 변화 시각화하기
color_map = {"멀티팩터": "black"}
fig = px.line(data_frame=total_assets, color_discrete_map=color_map)
fig.show()

In [56]:
metric_calculate(df=kospi_returns, fromdate=simulation_fromdate)

'CAGR: 0.07447742899224363, MDD: -0.31631897633315964, Sharpe Ratio: 0.5033032236063941, Sortino Ratio: 0.49128436907510104'